# Data cleaning and feature creation
Often writes data to data/processed or data/interim

In [1]:
import polars as pl
import os

DATA_PATH = "../data/raw"

In [2]:
# Load data
file_names = [x for x in os.listdir(DATA_PATH) if ".csv" in x]  # List all CSV files in the raw data directory

df_list = []
for file in file_names:
    df = pl.scan_csv(os.path.join(DATA_PATH, file))
    df_list.append(df)
    
# Combine lazyframes into a single lazyframe
df_combined = pl.concat(df_list, how="vertical")
df_combined_col = df_combined.collect()


In [3]:
# TBC: Clean Station names

In [4]:
# To proceed with only station IDs having at least 2000 dates
selected_stations_list = (df_combined_col.group_by("station_id")
 .agg(pl.col("date").n_unique().alias("date_unique_count"))
 .sort("date_unique_count", descending=True)
 .filter(pl.col("date_unique_count") >= 2500)
 .select(pl.col("station_id"))
).to_series().to_list()

: 

: 

In [ ]:
# Aggregate rainfall min max per day
df_agg = (df_combined_col.filter(pl.col("station_id").is_in(selected_stations_list))
          .group_by(["station_id", "date"])
          .agg(
    pl.min("reading_value").alias("min_rainfall"),
    pl.max("reading_value").alias("max_rainfall")
         ).sort("station_id", "date"))
df_agg.head()

In [ ]:
# Tag whether 